In [2]:
import sys
sys.path.append('..')

In [35]:
from utils.deepmatcher_utils import wrapDm
import deepmatcher as dm
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
from tqdm import tqdm

In [48]:
def getError(attribute_sets,df,study_matchings,model,lprefix='ltable_',rprefix='rtable_'):
    errors = {}
    originalProba = wrapDm(df,model,['label'])
    originalF1Score = f1_score(df.label.values,np.argmax(originalProba,axis=1))
    for attr_set in tqdm(attribute_sets):
        if study_matchings:
            df_to_alter = df[df.label==1].copy()
            unaltered_df = df[df.label==0].copy()
        else:
            df_to_alter = df[df.label==0].copy()
            unaltered_df = df[df.label==1].copy()
        for attr in attr_set:
            if study_matchings:
                df_to_alter[rprefix+attr] = ""
            else:
                df_to_alter[lprefix+attr] = ""
                df_to_alter[rprefix+attr] = ""
        currentProba = wrapDm(pd.concat([df_to_alter,unaltered_df],ignore_index=True),model,['label'])
        currentF1 = f1_score(df.label.values,np.argmax(currentProba,axis=1))
        errors[attr_set] = currentF1
    return errors

In [42]:
df = pd.read_csv('../datasets/Structured/Walmart-Amazon/merged_test.csv')
hybrid = dm.MatchingModel(attr_summarizer='hybrid')
hybrid.load_state('../models/walmartamazon_hybrid.pth')

In [50]:
attribute_sets = [('modelno',),('title',),('price',),('brand',),('category',)]
getError(attribute_sets,df,True,hybrid)





  0%|          | 0/5 [00:00<?, ?it/s]



 20%|██        | 1/5 [00:16<01:05, 16.33s/it]



 40%|████      | 2/5 [00:33<00:49, 16.53s/it]



 60%|██████    | 3/5 [00:49<00:33, 16.55s/it]



 80%|████████  | 4/5 [01:07<00:16, 16.94s/it]



100%|██████████| 5/5 [01:24<00:00, 16.93s/it]


{('modelno',): 0.05454545454545455,
 ('title',): 0.05042016806722689,
 ('price',): 0.051679586563307484,
 ('brand',): 0.05305039787798409,
 ('category',): 0.051948051948051945}

In [51]:
getError(attribute_sets,df,False,hybrid)





  0%|          | 0/5 [00:00<?, ?it/s]



 20%|██        | 1/5 [00:16<01:04, 16.18s/it]



 40%|████      | 2/5 [00:31<00:47, 15.89s/it]



 60%|██████    | 3/5 [00:48<00:32, 16.20s/it]



 80%|████████  | 4/5 [01:05<00:16, 16.43s/it]



100%|██████████| 5/5 [01:21<00:00, 16.39s/it]


{('modelno',): 0.21119324181626187,
 ('title',): 0.19218241042345274,
 ('price',): 0.20276497695852533,
 ('brand',): 0.2,
 ('category',): 0.19951338199513383}

In [52]:
df = pd.read_csv('../datasets/Structured/Amazon-Google/merged_test.csv')
hybrid_google = dm.MatchingModel(attr_summarizer='hybrid')
hybrid_google.load_state('../models/amazongoogle_hybrid.pth')

In [53]:
attribute_sets = [('title',),('price',),('manufacturer',)]
getError(attribute_sets,df,True,hybrid_google)





  0%|          | 0/3 [00:00<?, ?it/s]



 33%|███▎      | 1/3 [00:11<00:23, 11.53s/it]



 67%|██████▋   | 2/3 [00:22<00:11, 11.39s/it]



100%|██████████| 3/3 [00:34<00:00, 11.57s/it]


{('title',): 0.1615598885793872,
 ('price',): 0.13333333333333333,
 ('manufacturer',): 0.13257575757575757}